In [1]:
import uuid
import random
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

#  path modification to locate helpers module
import sys
import os
project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_path not in sys.path:
    sys.path.append(project_path)

from helpers import create_table_leanx as ctl, helpers
import values.values_default as values
from content_generators import master_data, text_data, sales_doc_data

In [2]:
# master tables
for method in (
     master_data.users,
     master_data.customers_and_vendors, 
     master_data.plants, 
     master_data.materials, 
     master_data.material_support,
     master_data.routes,
     master_data.company_codes
 ):
     table_dict = method()
     for k, v in table_dict.items():
         table_name = k.split('_')[0]
         all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
         df = pd.concat([all_cols, pd.DataFrame(v.values())])

         directory = os.path.dirname(f'../data/om/master/{table_name}.csv')
         if not os.path.exists(directory):
             os.makedirs(directory)

         df.to_csv(f'../data/om/master/{table_name}.csv', index=False)

/var/folders/1q/whzv5lqd6sjb2d4mnmydzgj40000gn/T/ipykernel_46167/3471590064.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([all_cols, pd.DataFrame(v.values())])
/var/folders/1q/whzv5lqd6sjb2d4mnmydzgj40000gn/T/ipykernel_46167/3471590064.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([all_cols, pd.DataFrame(v.values())])


In [3]:
# text tables
for method in (
    text_data.domain_fixed_values, 
    text_data.sales_doc_types, 
    text_data.sales_organizations,
    text_data.distribution,
    text_data.sales_doc_item_categories,
    text_data.sales_doc_rejection_reasons,
    text_data.system_status,
    text_data.blocking_reasons
):
    table_dict = method()
    for k, v in table_dict.items():
        table_name = k.split('_')[0]
        all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
        df = pd.concat([all_cols, pd.DataFrame(v.values())])

        directory = os.path.dirname(f'../data/om/text/{table_name}.csv')
        if not os.path.exists(directory):
            os.makedirs(directory)

        df.to_csv(f'../data/om/text/{table_name}.csv', index=False)

In [10]:
MARC = pd.read_csv('../data/om/master/MARC.csv')
MARA = pd.read_csv('../data/om/master/MARA.csv')
MAKT = pd.read_csv('../data/om/master/MAKT.csv')
KNB1 = pd.read_csv('../data/om/master/KNB1.csv')
KNA1 = pd.read_csv('../data/om/master/KNA1.csv')
LFB1 = pd.read_csv('../data/om/master/LFB1.csv')
LFA1 = pd.read_csv('../data/om/master/LFA1.csv')

all_prices = {}
all_availabilities = {}
all_material_groups = {**values.om_material_groups,**values.proc_material_groups}

# get price by MATNR
for nr in MARA['MATNR']:
    name = MAKT[MAKT['MATNR'] == nr]['MAKTX'].values[0]
    for k, v in all_material_groups.items():
        for mgrp, attr in v.items():
            for mat, details in attr['materials'].items():
                if name == mat:
                    all_prices[nr] = details['price']
                    all_availabilities[nr] = attr['availability']

def get_user_name(automation_probability: float):
    if random.random() <= automation_probability:
        return 'BATCH_JOB'
    return random.choice(list(values.om_users.keys()))

def get_time_consumption(start_date, planned_target_date, latest_date):
    return (latest_date - start_date) / (planned_target_date - start_date)

def generate_sequential_dates(total_dates=5000):
    # Define year probabilities and month probabilities for each year
    years = [2023, 2024]
    year_probs = np.array([0.5, 0.5]) / np.sum([0.5, 0.5])  # Normalize
    month_distributions = {
        2023: np.array([7, 5, 5, 4, 2, 6, 3, 5, 5, 6, 7, 10]),
        2024: np.array([7, 5, 5, 4, 2, 6, 3, 5, 8, 7, 9, 9]),
    }
    month_probs = {year: dist / dist.sum() for year, dist in month_distributions.items()}
    
    date_list = []
    while len(date_list) < total_dates:
        # Select a year based on probability
        year = int(np.random.choice(years, p=year_probs))
        # Select a month based on year-dependent probability
        month = np.random.choice(range(1, 13), p=month_probs[year])
        # Get the number of days in the selected month
        print(year, type(year), month, type(month))
        last_day = (datetime.date(year, month % 12 + 1, 1) - datetime.timedelta(days=1)).day
        # Generate all days in the selected month
        for day in range(1, last_day + 1):
            date_list.append(datetime.date(year, month, day))
            if len(date_list) >= total_dates:
                break  # Stop if we reach the desired count
    # Sort the dates to ensure they are in chronological order
    date_list.sort()
    return date_list

def get_params():
    # BASICS
    company_code = random.choices(list(values.om_company_codes.keys()),[10, 60, 65, 20, 75, 15, 55, 30, 35, 60, 15, 60, 13, 20, 15, 10, 35, 65],k=1)[0] #emphasis on EMEA company codes / plants
    plant = random.choice(values.om_company_codes[company_code]['plants'])
    konnr = str(uuid.uuid4())[-15:]
    lifnr = random.choice(list(LFB1[LFB1['BUKRS'] == company_code]['LIFNR']))
    requested_by = get_user_name(0.1)
    sales_org=random.choice(values.om_plants[plant]['sales_orgs'])
    purchasing_org=random.choice(values.om_plants[plant]['sales_orgs'])
    sales_office=random.choice(list(values.om_sales_orgs[sales_org]['sales_offices'].keys()))
    doc_type_number = random.random()
    sales_doc_type = ('ZDIR' if (doc_type_number < 0.4) else
                      'ZOR' if (doc_type_number < 0.8) else 
                      random.choice(list(values.om_sales_doc_types.keys()))) 
    mapping_salesdoc = {'ZOR': '40', 'ZDLR': '20', 'ZDIR': '10', 'ZDIS': '30','ZEXP': '50'}
    distribution_channel=mapping_salesdoc[sales_doc_type]
    kunnr = random.choice(list(KNB1[KNB1['BUKRS'] == company_code]['KUNNR']))
    customer_name = KNA1[KNA1['KUNNR'] == kunnr]['NAME1'].values[0]
    payment_term = random.choice(list(KNB1[KNB1['KUNNR'] == kunnr]['ZTERM']))
    credit_risk = values.om_customers[customer_name]['credit_risk']
    automation_rate = values.om_sales_orgs[sales_org]['Automation_rate'][distribution_channel]
    requested_dev_date = random.randint(7, 17)
    targeted_plants = ('PL03','PL05', 'PL10') # for late billing and unbilled
    has_invoice = False if plant in targeted_plants and random.random() > 0.94 else True  # for unbilled
    has_late_invoice = True if has_invoice == True and plant in targeted_plants else False # for late billing emphasis

#--------------------------------------------------------------------------------
    #MATERIALS : materials, quantities, prices, availabilities, delivery status 
    all_matnrs = MARC[MARC['WERKS'] == plant]['MATNR'].unique()
    matnrs = random.sample(list(all_matnrs[0:-45]), min(random.randint(5, 25), len(all_matnrs))) #OM materials
    blocked_matnrs = random.sample(range(len(matnrs)), random.randint(4, len(matnrs)))
    proc_matnrs = random.sample(list(all_matnrs[-45:]), min(random.randint(5, 16), len(all_matnrs)))  #PROC materials 
    quantity_factor = random.randint(5, 7) if values.om_plants[plant]['high_value'] else random.randint(2, 4)
    quantities = [random.randint(3, 6)*quantity_factor for temp_quantity_index in range(len(matnrs))]  # OM quantities
    proc_quantities = [random.randint(1, 4) for m in range(len(proc_matnrs))]  #PROC quantities
    prices = [] #OM prices
    availabilities = [] #OM availabilities
    delivery_status_boundaries = []
    item_delivery_status = []
    for i in range(len(matnrs)):
        prices.append(all_prices[matnrs[i]]*0.001) # TEST slashing prices by 90% 
        availabilities.append(all_availabilities[matnrs[i]])
        a = all_availabilities[matnrs[i]]
        #[Probability of being late, Prabability of being late + Prob of being on Time]
        # Prob of late = 0.7-0.65*a, Prob of On-Time = 0.25+0.55*a, Prob of Early = 0.05 +0.1*a
        # -> Boundaries CASE WHEN random number < Prob of late Then LATE, CASE WHEN random number > Prob of Late AND <Prob of Late + Prob of ON-Time THEN ON-Time ELSE Early END
        delivery_status_boundaries.append([0.9-0.7*a,0.95-0.1*a])
    proc_prices = [all_prices[proc_matnrs[temp_matnr_index]] for temp_matnr_index in range(len(proc_matnrs))] #PROC prices
    #delivery st
    for i in range(len(delivery_status_boundaries)):
        late_bound, ot_bound = delivery_status_boundaries[i][0], delivery_status_boundaries[i][1]
        r = random.random()
        # if r < late_bound or (r < 0.4 and requested_dev_date <= 11):
        if (r < 0.7 and requested_dev_date <= 11):
            item_delivery_status.append({'status': 'late', 'prob': round(r,2)})
        elif (r < 0.4 and requested_dev_date >= 11):
            item_delivery_status.append({'status': 'late', 'prob': round(r,2)})
        elif r < late_bound:
            item_delivery_status.append({'status': 'late', 'prob': round(r,2)})
        elif r >= late_bound and r<ot_bound:
            item_delivery_status.append({'status': 'ot', 'prob': round(r-late_bound,2)})
        else:
            item_delivery_status.append({'status': 'early', 'prob': round(r-ot_bound,2)})
    total = np.sum([prices[i]*quantities[i] for i in range(len(prices))])
# --------------------------------------------------------------------------

    # TIME DISTRIBUTION
    pr_req_years = [2023, 2024]
    pr_req_year_probability = np.array([0.5, 0.5])
    pr_req_year_probability = pr_req_year_probability / pr_req_year_probability.sum(axis=0, keepdims=True)
    pr_req_year = pr_req_years[np.random.choice(2, p=pr_req_year_probability)]
    if pr_req_year == 2023:
        pr_req_month_distribution = np.array([7, 5, 5, 4, 2, 6, 3, 5, 5, 6, 7, 10])
        pr_req_month_probability = pr_req_month_distribution / pr_req_month_distribution.sum(axis=0, keepdims=True)
        pr_req_month = np.random.choice(12, p=pr_req_month_probability) + 1 # because months should be in [1, 12] for datetime
    else:
        pr_req_month_distribution = np.array([7, 5, 5, 4, 2, 6, 3, 5, 8, 7, 9, 9])
        pr_req_month_probability = pr_req_month_distribution / pr_req_month_distribution.sum(axis=0, keepdims=True)
        pr_req_month = np.random.choice(12, p=pr_req_month_probability) + 1 # because months should be in [1, 12] for datetime
    
    params = {
        'kunnr': kunnr,
        'konnr': konnr,
        'lifnr': lifnr,
        'credit_risk': credit_risk,
        'payment_term': payment_term,
        'company_code': company_code,
        'plant': plant,
        'matnrs': matnrs,
        'blocked_matnrs': blocked_matnrs,
        'proc_matnrs': proc_matnrs,
        'quantities': quantities,
        'proc_quantities': proc_quantities,
        'prices': prices,
        'proc_prices': proc_prices,
        'sales_org': sales_org,
        'purchasing_org': purchasing_org,
        'sales_office': sales_office,
        'distribution_channel': distribution_channel,
        'availabilities': availabilities,
        'sales_doc_type': sales_doc_type,
        'total': total,
        'automation_rate': automation_rate,
        'item_delivery_status': item_delivery_status,
        'pr_req_year': pr_req_year,
        'pr_req_month': pr_req_month,
        'requested_dev_date' : requested_dev_date,
        'requested_by': requested_by,
        'has_invoice': has_invoice, 
        'has_late_invoice': has_late_invoice,
    }

    return params

In [11]:
# sales order tables
sales_doc_tables = {
    'BKPF_json': {},
    'BSEG_json': {},
    'CDHDR_json': {},
    'CDPOS_json': {},
    'EKBE_json': {}, 
    'EKKO_json':{},
    'EKPO_json':{},
    'JCDS_json': {},
    'LIKP_json': {},
    'LIPS_json': {},
    'MSEG_json': {},
    'NAST_json': {},
    'VBAK_json': {},
    'VBAP_json': {},
    'VBEP_json': {},
    'VBFA_json': {},
    'VBKD_json': {},
    'VBRK_json': {},
    'VBRP_json': {},
    'VBUK_json': {},
    'VTTK_json':{},
    'VTTP_json':{},
}

number_of_cases = 500

dates = generate_sequential_dates(number_of_cases)

for i in range(number_of_cases):
    transition_prob = np.array([
      # [0     1     2     3     4     5     6     7     8     9     10    11    12    13    14    15    16    17    18  ]  # STarts at Create Sales Order
        [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05], # 0 Approve Sales Order
        [0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.01], # 1 Generate Delivery Document
        [0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05], # 2 Pick Items
        [0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.01], # 3 Post Goods Issue / Create Shipment
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05], # 4 Send Invoice
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.01], # 5 Receive Delivery Confirmation
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # 6 Clear Invoice
         
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # 7 Reject Sales Order
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.75, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.25], # 8 Set Credit Block
        [0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.10], # 9 Release Credit Block
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.65, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.35], # 10 Set Sales Order Delivery Block
        [0.00, 0.00, 0.90, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.10], # 11 Release Sales Order Delivery Block
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.05], # 12 Set Sales Order Billing Block
        [0.00, 0.00, 0.00, 0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 13 Release Sales Order Billing Block
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.61, 0.00, 0.00, 0.05], # 14 Set Customer Billing / Delivery Block
        [0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 15 Release Customer Billing / Delivery Block
        [0.00, 0.95, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.01], # 16 Change Payment Term
        [1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00], # 17 Change Quantity / Price

        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # 18 Terminate

    ])

    params=get_params()
    #latest_date = datetime(params['pr_req_year'], params['pr_req_month'], random.randint(1, 28)).date()
    latest_date = dates[i]
    so_created_date = latest_date
    latest_time = helpers.generate_random_time()
    so_created_by = get_user_name(0.5*params['automation_rate'])
    reqested_delivery_date= latest_date + timedelta(days = params['requested_dev_date'])
    # print(params['total'])

    sd = sales_doc_data.SalesAndDistribution(
        vbeln=f'{str(uuid.uuid4())[-11:]}',
        params=params,
        start_date=latest_date,
        index=i
    )

    #Purchase Order tables EKKO, EKPO, NAST -> irrelevant for now 
    sd.create_purchase_order(
        aedat=latest_date,
        ernam=get_user_name(0.4*params['automation_rate']),
        utime=latest_time
    )
    latest_date = helpers.add_random_days(1, 3, latest_date)
    sd.send_purchase_order(
            usnam=get_user_name(0.4*params['automation_rate']),
            erdat=latest_date
        )
    #Purchase Order tables END

    sd.create_sales_order(
        reqested_delivery_date=reqested_delivery_date,
        shipping_condition=random.choice(list(values.shipping_conditions.keys())),
        erdat=latest_date,
        ernam=so_created_by,
        atime=latest_time
    )
    latest_time = helpers.add_random_hours(1, latest_time)
    

    # Define the deviation of days between scheduled issue date VBEP.EDATU and actual issue date MSEG.CPUDT_MKPF
    delivery_date_deviation = []
    for i in range(len(params['matnrs'])):
        days_deviation = helpers.UPTO_MONTH()*params['item_delivery_status'][i]['prob']
        days_temp = round(days_deviation.total_seconds()/(24*3600))
        days_deviation = days_temp #timedelta(days=days_temp)
        if params['item_delivery_status'][i]['status'] == 'late':
            scheduled_date = min(-1, -days_deviation)
        elif params['item_delivery_status'][i]['status'] == 'early':
            scheduled_date  = max(1, days_deviation)
        else:
            scheduled_date = 0
        delivery_date_deviation.append(scheduled_date)   

    # decide on approve or reject sales order
    def nex_step():
        stepy = 0
        if so_created_by != 'BATCH_JOB':
            if random.random() < 0.11: # adjusted for value framing. 
                stepy = 7 # Reject Sales Order for inaccurate information
        elif np.average(params['availabilities']) < 0.85:
            if random.random() < 0.55:
                stepy = 17 # Quantity Change
        return stepy # Approve Reject or Change
    
    step = nex_step()
    while step != 18: # until termination
        if step == 0:
            latest_date = helpers.add_random_days(1, 1, latest_date)
            sd.approve_sales_order(
                usnam=get_user_name(0.9*params['automation_rate']),
                udate=latest_date,
                atime=latest_time
            )
            
            # Set Credit Block
            if params['credit_risk'] > 0.75:
                transition_prob[step][8] = 0.5
                latest_date = helpers.add_random_days(1, 3, latest_date)
            # Customer Billing / Delivery Block
            elif params['credit_risk'] > 0.6:
                transition_prob[step][14] = 0.5
                latest_date = helpers.add_random_days(1, 3, latest_date)
            # Change Payment Term
            elif random.random() > 0.81:
                transition_prob[step][16] = 0.5
                latest_date = helpers.add_random_days(1, 3, latest_date)
            else:
                latest_date = helpers.add_random_days(1, 1, latest_date)
        elif step == 1:
            sd.generate_delivery_document(
                ernam=get_user_name(0.9*params['automation_rate']), 
                erdat=latest_date,
                planned_delivery_date=latest_date + helpers.UPTO_WEEK(),
                picking_date=None,
                delivery_date=None,
                confirmation_date=None,
                atime=latest_time
            )
            latest_time = helpers.add_random_hours(3, latest_time)

            # Delivery blocks
            if np.average(params['availabilities']) < 0.6:
                latest_time = helpers.add_random_hours(1, latest_time)
                transition_prob[step][10] = 0.5
            elif np.average(params['availabilities']) < 0.8 and params['has_late_invoice'] == True:
                latest_time = helpers.add_random_hours(1, latest_time)
                transition_prob[step][10] = 0.7
            else:
                latest_time = helpers.add_random_hours(1, latest_time)
        elif step == 2:
            sd.pick_items(
                usnam=get_user_name(0.4*params['automation_rate']), 
                udate=latest_date,
                atime=helpers.generate_random_time()
            )
            latest_time = helpers.add_random_hours(8, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 3:
            latest_time = helpers.add_random_hours(8, latest_time)
            latest_date = helpers.add_random_days(3, 6, latest_date)
            sd.post_goods_issue(
                cpudt=latest_date,
                usnam=get_user_name(0.6*params['automation_rate']),
                delivery_date_deviation = delivery_date_deviation,
                atime=helpers.generate_random_time()         
            )
            
# ------ Uncomment snippet to understand early and late deliveries
            # item_status_early = {"Early":0, "Early works": 0}
            # item_status_late = {"Late":0, "Late works": 0}
            # item_status_ot = {"ot":0, "ot works": 0}

            # for i, d in enumerate(delivery_date_deviation):
            #     if d>0:
            #         item_status_early["Early"]+=1
            #         if latest_date+timedelta(days=delivery_date_deviation[i]) > (so_created_date + timedelta(days=1)):
            #             item_status_early["Early works"]+=1
            #         else:
            #             print(f'Deviation from delivery date is {d} days. Min Date is {so_created_date + timedelta(days=1)} new date should be {latest_date+timedelta(days=d)}, send goods date {latest_date}')
            #     elif d<0:
            #         item_status_late["Late"]+=1
            #         if latest_date+timedelta(days=delivery_date_deviation[i]) > (so_created_date + timedelta(days=1)):
            #             item_status_late["Late works"]+=1
            #         else:
            #             print(f'Deviation from delivery date is {d} days. Min Date is {so_created_date + timedelta(days=1)} new date should be {latest_date+timedelta(days=d)}, send goods date {latest_date}')
            #     else:
            #         item_status_ot["ot"]+=1
# ----------------------------------------------------------------------------------

        # Sales Order Billing Block
            if params['total'] > 450:  # normally 450000 but adjusted for TEST adjust with multiplcation factor on material prices in params. 
                transition_prob[step][12] = 0.3
                latest_time = helpers.add_random_hours(12, latest_time)
                # latest_date = helpers.add_random_days(0, reqested_delivery_date.day, latest_date)
                latest_date = helpers.add_random_days(0, 7, latest_date)   
            else:
                latest_time = helpers.add_random_hours(2, latest_time)
                # latest_date = helpers.add_random_days(0, reqested_delivery_date.day, latest_date)
                latest_date = helpers.add_random_days(0, 7, latest_date)
        elif step == 4:
            if params['has_invoice'] == True:
                sd.create_invoice(
                    ernam=get_user_name(0.8*params['automation_rate']),
                    erdat=latest_date if params['has_late_invoice'] == False else helpers.add_random_days(5, 26, latest_date),
                    atime=helpers.generate_random_time()
                )
                latest_time = helpers.add_random_hours(2, latest_time)
                latest_date = helpers.add_random_days(0, 7, latest_date)
        elif step == 5:
            sd.delivery_confirmation(
                usnam=get_user_name(0.8*params['automation_rate']), 
                udate=latest_date
            )
            latest_time = helpers.add_random_hours(1, latest_time)
        
            if get_time_consumption(start_date=so_created_date, planned_target_date=reqested_delivery_date, latest_date=latest_date) > 1.25:
                latest_date = helpers.add_random_days(0, 14, latest_date)
            else:
                latest_date = helpers.add_random_days(0, 7, latest_date)
        elif step == 6:
            if params['has_invoice'] == True:
                sd.clear_debit_invoice(
                    cpudt=latest_date,
                    usnam=get_user_name(0.4*params['automation_rate']),
                    cleared_date=latest_date + helpers.UPTO_WEEK(),
                    atime=helpers.generate_random_time()
                )
                latest_time = helpers.add_random_hours(2, latest_time)
                latest_date = helpers.add_random_days(0, 0, latest_date)
        
        elif step == 7:
            latest_date = helpers.add_random_days(1, 1, latest_date)
            sd.reject_sales_order(
                udate=latest_date,
                usnam=get_user_name(0.9*params['automation_rate']), 
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 8:
            latest_date = helpers.add_random_days(1, 1, latest_date)
            sd.set_credit_block(
                udate=latest_date,
                usnam=get_user_name(0.55*params['automation_rate'])
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(1, 15, latest_date)
        elif step == 9:
            sd.release_credit_block(
                udate=latest_date,
                usnam=get_user_name(0.2*params['automation_rate']), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 10:
            sd.set_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.55*params['automation_rate']), 
            )
            latest_time = helpers.add_random_hours(2, latest_time)
            latest_date = helpers.add_random_days(1, 30, latest_date)
        elif step == 11:
            sd.release_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.2*params['automation_rate']), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 12:
            sd.set_sales_order_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.6*params['automation_rate']), 
                #blocked_matnrs=[params['matnrs'][0]]
                blocked_matnrs= params['blocked_matnrs']
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(1, 7, latest_date)
        elif step == 13:
            sd.release_sales_order_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.2*params['automation_rate']), 
                #blocked_matnrs=[params['matnrs'][0]]
                blocked_matnrs= params['blocked_matnrs']
            )
            latest_time = helpers.add_random_hours(4, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 14:
            if params['credit_risk'] > 0.67:
                sd.set_customer_billing_block(
                    udate=latest_date,
                    usnam=get_user_name(0.4*params['automation_rate']), 
                )
            else:
                sd.set_customer_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.4*params['automation_rate']), 
                )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 14, latest_date)
        elif step == 15:
            if params['credit_risk'] > 0.67:
                sd.release_customer_billing_block(
                udate=latest_date,
                usnam=get_user_name(0.3*params['automation_rate']), 
                )
            else:
                sd.release_customer_delivery_block(
                udate=latest_date,
                usnam=get_user_name(0.3*params['automation_rate']),
                )
            latest_time = helpers.add_random_hours(5, latest_time)
            latest_date = helpers.add_random_days(0, 0, latest_date)
        elif step == 16:
            sd.change_payment_term(
                udate=latest_date,
                usnam=get_user_name(0.2*params['automation_rate']), 
            )
            latest_time = helpers.add_random_hours(1, latest_time)
            latest_date = helpers.add_random_days(0, 7, latest_date)

        elif step == 17:
            latest_date = helpers.add_random_days(1, 1, latest_date)
            if random.random() < 0.5:
                old_quantities = params['quantities']
                new_quantity_lines = random.sample(range(len(old_quantities)), random.randint(0, len(old_quantities)))
                new_quanity_quantities = [round(old_quantities[j]*(1+random.random())) for j in new_quantity_lines]
                sd.change_quantity( # likely goes to 'Send PO'
                    udate=latest_date,
                    usnam=get_user_name(0.1*params['automation_rate']),
                    line_numbers=new_quantity_lines,
                    line_quantities=new_quanity_quantities
                )
            else:
                old_prices = params['prices']
                new_price_lines = random.sample(range(len(old_prices)), random.randint(0, len(old_prices)))
                new_prices = [round(old_prices[j]*(random.uniform(0.5, 1.5)), 2) for j in new_price_lines]
                sd.change_price( # likely goes to 'Send PO'
                    udate=latest_date,
                    usnam=get_user_name(0.1*params['automation_rate']),
                    line_numbers=new_price_lines,
                    line_prices=new_prices
                ) 
            latest_date += helpers.UPTO_WEEK()
        # update transition matrix to sum to 1
        transition_prob = transition_prob / transition_prob.sum(axis=1, keepdims=True)
        step = np.random.choice(19, p=transition_prob[step])

    for k, v in sd.tables.items():
        for entry_key in list(v.keys()):
            sales_doc_tables[k][entry_key] = sd.tables[k][entry_key]

2023 <class 'int'> 6 <class 'numpy.int64'>


TypeError: descriptor 'date' for 'datetime.datetime' objects doesn't apply to a 'int' object

In [ ]:
for table, rows in sales_doc_tables.items():
    table_name = table.split('_')[0]
    all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
    all_cols_req = ctl.clean_columns(all_cols,table_name)
    df = pd.concat([all_cols_req, pd.DataFrame(rows.values())])

    directory = os.path.dirname(f'../data/om/sales-document/{table_name}.csv')
    if not os.path.exists(directory):
        os.makedirs(directory)

    df.to_csv(f'data/om/sales-document/{table_name}.csv', index=False)